In [ ]:
!pip install telethon requests qrcode Pillow nest_asyncio opencv-python


In [ ]:
# Установка необходимых библиотек
!pip install telethon requests nest_asyncio

# Импорт библиотек
import os
import json
import requests
import asyncio
from telethon import TelegramClient
from telethon.tl.types import MessageMediaPhoto, MessageMediaDocument, MessageMediaWebPage
import nest_asyncio

# Применение nest_asyncio для корректной работы
nest_asyncio.apply()

# --- Ваши данные из GitHub Secrets ---
api_id = int(os.environ['API_ID'])
api_hash = os.environ['API_HASH']
phone = os.environ['PHONE_NUMBER']
n8n_webhook_url = os.environ['N8N_WEBHOOK_URL']
# ---------------------------------------------

# --- Список каналов для парсинга ---
# Убедитесь, что каналы указаны правильно
channel_names = ['@sochitypical' , '@chp_sochi']
# ---------------------------------------------

async def main():
    # Подключение к Telegram
    client = TelegramClient('my_session', api_id, api_hash)
    await client.connect()

    # Проверка авторизации
    if not await client.is_user_authorized():
        print("Ошибка: не удалось авторизоваться. Убедитесь, что файл my_session.session загружен в репозиторий.")
        return

    # Получение последних 5 сообщений из каждого канала в списке
    results = []
    for channel_name in channel_names:
        try:
            async for msg in client.iter_messages(channel_name, limit=5):
                # Проверяем, есть ли в сообщении хоть что-то (текст ИЛИ медиа)
                if msg.text or msg.media:

                    # Создаем базовый объект для сообщения
                    message_data = {
                        'channel': channel_name,
                        'message_id': msg.id, # Используйте message_id для скачивания в n8n
                        'text': msg.text if msg.text else '',
                        'date': str(msg.date),
                        'has_media': False,
                        'media_type': None,
                        'file_name': None,
                    }

                    # Добавляем информацию о медиа
                    if msg.media:
                        message_data['has_media'] = True
                        message_data['media_type'] = type(msg.media).__name__

                        # Пытаемся получить имя файла
                        if isinstance(msg.media, MessageMediaDocument) and msg.media.document.attributes:
                            for attr in msg.media.document.attributes:
                                if hasattr(attr, 'file_name'):
                                    message_data['file_name'] = attr.file_name
                                    break

                    results.append(message_data)

        except Exception as e:
            print(f"Ошибка при парсинге канала {channel_name}: {e}")
            continue

    # Отправка данных в n8n
    try:
        requests.post(n8n_webhook_url, json=results)
        print(f"Данные успешно отправлены в n8n. Отправлено {len(results)} сообщений.")
    except Exception as e:
        print(f"Ошибка при отправке данных в n8n: {e}")

    # Отключение от Telegram
    await client.disconnect()

if __name__ == '__main__':
    asyncio.run(main())

Данные успешно отправлены в n8n.
